# Code that might be useful later

## Sklearn trick for clustering strings

In [2]:
class SklearnStringTrick:
    """
    String clustering trick shown at:
        https://scikit-learn.org/0.16/faq.html#how-do-i-deal-with-string-data-or-trees-graphs

    Class structure allows for method reuse over distinct data

    """

    def __init__(self, dataset=[]):
        self.dataset = dataset

    def lev_dist_sklearn_urls(self, x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        return levenshtein(self.dataset[i], self.dataset[j])

In [3]:
string_trick = SklearnStringTrick()

## Pattern for filtering IPv6 addresses

## My attempt for regex splitting domain and subdomains

## Alternative library for tld matching

#### N-grams

#### Non-functional version of TF-IDF

Crashes on the .toarray() function
Might be memory issue?

TfidfVectorizer works though

## TF-IDF using HashingVectorizer

Not working yet

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer


class IncrementalHashingVectorizer:
    def __init__(self, n_features=2**18, ngram_range=(1, 1), **kwargs):
        self.vectorizer = HashingVectorizer(
            n_features=n_features, ngram_range=ngram_range, **kwargs
        )
        self.partial_fit_calls = 0

    def partial_fit(self, X):
        if self.partial_fit_calls == 0:
            self.features_sum = np.zeros(self.vectorizer.n_features)
            self.feature_counts = np.zeros(self.vectorizer.n_features)

        transformed = self.vectorizer.transform(X)
        self.features_sum += transformed.sum(axis=0)
        self.feature_counts += transformed.getnnz(axis=0)
        self.partial_fit_calls += 1
        return self

    def transform(self, X):
        transformed = self.vectorizer.transform(X)
        idf_vector = np.log(self.partial_fit_calls / (1 + self.feature_counts)) + 1
        return transformed.multiply(idf_vector)


incremental_vectorizer = IncrementalHashingVectorizer(
    ngram_range=NGRAM_RANGE, analyzer="char"
)


incremental_vectorizer.partial_fit([doc])

transformed_documents = incremental_vectorizer.transform(documents)

## TF-IDF on batches

In [ ]:
# Create TfidfVectorizer instance for domain column
tfidf_domain = TfidfVectorizer(analyzer="char", ngram_range=NGRAM_RANGE)
# Fit and transform domain column
tfidf_domain_matrix = tfidf_domain.fit_transform(urls_df["domain"])

# Create TfidfVectorizer instance for subdomain column
tfidf_subdomain = TfidfVectorizer(analyzer="char", ngram_range=NGRAM_RANGE)
# Fit and transform subdomain column
tfidf_subdomain_matrix = tfidf_subdomain.fit_transform(urls_df["subdomain"])

# Create pandas dataframe from tfidf_matrix
tfidf_domain_df_tfidfv = pd.DataFrame(
    tfidf_domain_matrix.toarray(),
    columns=tfidf_domain.get_feature_names_out(),
    index=urls_df.index,
)
tfidf_subdomain_df_tfidfv = pd.DataFrame(
    tfidf_subdomain_matrix.toarray(),
    columns=tfidf_subdomain.get_feature_names_out(),
    index=urls_df.index,
)

# Concatenate the dataframes
# tfidf_df = pd.concat([tfidf_domain_df, tfidf_subdomain_df], axis=1)

In [ ]:
tfidf_domain_df_tfidfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638971 entries, 0 to 638970
Columns: 165105 entries, '01 to zzzi
dtypes: float64(165105)
memory usage: 786.0 GB


In [ ]:
tfidf_subdomain_df_tfidfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638971 entries, 0 to 638970
Columns: 162855 entries, '00 to zzzz
dtypes: float64(162855)
memory usage: 775.3 GB


## How to use LSA for TF-IDF

In [27]:
# Taken from https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#performing-dimensionality-reduction-using-lsa

lsa = make_pipeline(TruncatedSVD(n_components=10), Normalizer(copy=False))

X_domain_lsa = lsa.fit_transform(X_domain_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

Explained variance of the SVD step: 49.2%
